# Clonamos el repositorio para obtener los dataSet





In [20]:
!git clone https://github.com/joanby/ia-course.git

fatal: destination path 'ia-course' already exists and is not an empty directory.


# Damos acceso a nuestro Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Test it

In [ ]:
!ls '/content/drive/My Drive' 

#Google colab tools

In [21]:
from google.colab import files # Para manejar los archivos y, por ejemplo, exportar a su navegador
import glob # Para manejar los archivos y, por ejemplo, exportar a su navegador
from google.colab import drive # Montar tu Google drive

# Instalar dependencias de Renderizado, tarda alrededor de 45 segundos


In [22]:
!apt-get update > /dev/null 2>&1
!apt-get install python-opengl -y > /dev/null 2>&1
!apt install xvfb -y --fix-missing > /dev/null 2>&1
!apt-get install ffmpeg > /dev/null 2>&1
!apt-get install x11-utils > /dev/null 2>&1
!apt-get install pyglet > /dev/null 2>&1


# Instalar OpenAi Gym

In [23]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install piglet > /dev/null 2>&1
!pip install 'gym[box2d]' > /dev/null 2>&1
#por si quieres algun environment en concreto
#!pip install atari_py > /dev/null 2>&1
#!pip install gym[atari] > /dev/null 2>&1

# Todos los imports necesarios en google colab y helpers para poder visualizar OpenAi

In [24]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

# Activamos una vista, seria como crear un plot de una grafica en python

In [25]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900)) #Puedes modificar el high and width de la pantalla
display.start()

# Este código crea una pantalla virtual para dibujar imágenes del juego. 
## Si se ejecuta localmente, simplemente ignóralo

In [26]:
import os
if type(os.environ.get('DISPLAY')) is not str or \
        len(os.environ.get('DISPLAY')) == 0:
    !bash ../xvfb start
    %env DISPLAY=:1

# Funciones de utilidad para permitir la grabación de video del ambiente del gimnasio y su visualización
## Para habilitar la visualizacion por pantalla , tan solo haz "**environment = wrap_env(environment)**", por ejemplo: **environment = wrap_env(gym.make("MountainCar-v0"))**

In [27]:
import io
import glob
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from pathlib import Path

def show_one_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)

        content = ipythondisplay.display(HTML(data='''
        <video alt="test" autoplay loop controls style="height: 400px;">
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>
        '''.format(encoded.decode('ascii'))))
    else: 
        print("Couldn't find video")

def show_videos(video_path='video', prefix=''):
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

def wrap_env(env):
    env = gym.wrappers.Monitor(env, './video', force=True)
    return env

# Instalar pytorch

In [28]:
!pip install torch torchvision
!pip install numpy

# Nuestro Script

En google colab, para importar modulos se hace de manera diferente, ya que los notebooks que crea se guardan en una ruta temporal para despues guardarse en drive, por lo que si quieres cargar modulos externos se tiene que hacer:  
**import sys**   
**sys.path.append('path-carpeta')**  *#esto añade como una caché de rutas*  
*from module import function or class*

In [29]:
import sys
sys.path.append('/content/ia-course/tema3/libs/')
sys.path.append('/content/ia-course/tema3/utils/')
sys.path.append('/content/ia-course/tema3/environments/')

import torch
import numpy as np
from perceptron import SLP
from decay_schedule import LinearDecaySchedule
import random
import gym
from experience_memory import ExperienceMemory, Experience

MAX_NUM_EPISODES = 100000
STEPS_PER_EPISODE = 300

In [30]:
class SwallowQLearner(object):
    def __init__(self, environment, learning_rate = 0.005, gamma = 0.98):
        self.obs_shape = environment.observation_space.shape
        
        self.action_shape = environment.action_space.n
        self.Q = SLP(self.obs_shape, self.action_shape)
        self.Q_optimizer = torch.optim.Adam(self.Q.parameters(), lr = learning_rate)
        
        self.gamma = gamma
        
        self.epsilon_max = 1.0
        self.epsilon_min = 0.05
        self.epsilon_decay = LinearDecaySchedule(initial_value = self.epsilon_max,
                                                 final_value = self.epsilon_min, 
                                                 max_steps = 0.5 * MAX_NUM_EPISODES * STEPS_PER_EPISODE)
        self.step_num = 0
        self.policy = self.epsilon_greedy_Q
        
        self.memory = ExperienceMemory(capacity = int(1e5))
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        
         
    def get_action(self, obs):
        return self.policy(obs)
    
    def epsilon_greedy_Q(self, obs):
        if random.random() < self.epsilon_decay(self.step_num):
            action = random.choice([a for a in range(self.action_shape)])
        else:
            action = np.argmax(self.Q(obs).data.to(torch.device(self.device)).numpy())   
        self.step_num += 1 ##EN EL VIDEO SE NOS OLVIDÓ SUBIR EL STEP EN UNA UNIDAD
        return action
        
        
    def learn(self, obs, action, reward, next_obs):
        td_target = reward + self.gamma * torch.max(self.Q(next_obs))
        td_error = torch.nn.functional.mse_loss(self.Q(obs)[action], td_target)
        self.Q_optimizer.zero_grad()
        td_error.backward()
        self.Q_optimizer.step()
        
    def replay_experience(self, batch_size):
        """
        Vuelve a jugar usando la experiencia aleatoria almacenada
        :param batch_size: Tamaño de la muestra a tomar de la memoria
        :return: 
        """
        experience_batch = self.memory.sample(batch_size)
        self.learn_from_batch_experience(experience_batch)   
      
    def learn_from_batch_experience(self, experiences):
        """
        Actualiza la red neuronal profunda en base a lo aprendido en el conjunto de experiencias anteriores
        :param experiences: fragmento de recuerdos anteriores
        :return: 
        """
        batch_xp = Experience(*zip(*experiences))
        obs_batch = np.array(batch_xp.obs)
        action_batch = np.array(batch_xp.action)
        reward_batch = np.array(batch_xp.reward)
        next_obs_batch = np.array(batch_xp.next_obs)
        done_batch = np.array(batch_xp.done)
        
        td_target = reward_batch + ~done_batch * \
        np.tile(self.gamma, len(next_obs_batch)) * \
        self.Q(next_obs_batch).detach().max(1)[0].data.numpy()
        td_target = torch.from_numpy(td_target)
        
        td_target = td_target.to(self.device)
        action_idx = torch.from_numpy(action_batch).to(self.device)
        td_error = torch.nn.functional.mse_loss(
                self.Q(obs_batch).gather(1,action_idx.view(-1,1).long()),
                td_target.float().unsqueeze(1))
        
        self.Q_optimizer.zero_grad()
        td_error.mean().backward()
        self.Q_optimizer.step()
        

# Main

In [31]:
if __name__ == "__main__":
    environment = gym.make("CartPole-v0")
    agent = SwallowQLearner(environment)
    first_episode = True
    episode_rewards = list()
    for episode in range(MAX_NUM_EPISODES):
        obs = environment.reset()
        total_reward = 0.0
        for step in range(STEPS_PER_EPISODE):
            #environment.render()
            action = agent.get_action(obs)
            next_obs, reward, done, info = environment.step(action)
            agent.memory.store(Experience(obs, action, reward, next_obs, done))
            agent.learn(obs, action, reward, next_obs)
            
            obs = next_obs
            total_reward += reward
            
            if done is True:
                if first_episode:
                    max_reward = total_reward
                    first_episode = False
                episode_rewards.append(total_reward)
                if total_reward > max_reward:
                    max_reward = total_reward
                print("\nEpisodio#{} finalizado con {} iteraciones. Recompensa = {}, Recompensa media = {}, Mejor recompensa = {}".
                      format(episode, step+1, total_reward, np.mean(episode_rewards), max_reward))
                if agent.memory.get_size()>100:
                    agent.replay_experience(32)
                break
    environment.close()
    show_videos()


Episodio#0 finalizado con 26 iteraciones. Recompensa = 26.0, Recompensa media = 26.0, Mejor recompensa = 26.0

Episodio#1 finalizado con 33 iteraciones. Recompensa = 33.0, Recompensa media = 29.5, Mejor recompensa = 33.0

Episodio#2 finalizado con 35 iteraciones. Recompensa = 35.0, Recompensa media = 31.333333333333332, Mejor recompensa = 35.0

Episodio#3 finalizado con 13 iteraciones. Recompensa = 13.0, Recompensa media = 26.75, Mejor recompensa = 35.0


RuntimeError: ignored